In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)

spark = SparkSession \
    .builder \
        .appName("invalid_values") \
            .config("spark.some.config.option", "some-value") \
                .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/08 14:16:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import trim, col, row_number, monotonically_increasing_id
from pyspark.sql.window import Window

# Load the data stored in the file
df = spark.read.csv("dataset/2010_-_2016_School_Safety_Report_20240413.csv", header=True, inferSchema=True)

# Trim the leading and trailing whitespaces
for column in df.columns:
    df = df.withColumn(column, trim(col(column)))

# Add a unique index to each row
df_with_index = df.withColumn('INDEX', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

df_with_index.show()

24/05/08 14:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-------------+------+--------------------+-------------+--------------------+-------+--------------------------+--------+--------------------+---------+--------------------+-------+-----+--------+------+-----+--------+---------+------------+----------+-------------+-----------+----------+------------+--------+---------+----------+---------------+-----------------+------------+-------+----------+--------------------+-----+
|School Year|Building Code|   DBN|       Location Name|Location Code|             Address|Borough|Geographical District Code|Register|       Building Name|# Schools| Schools in Building|Major N|Oth N|NoCrim N|Prop N|Vio N|ENGroupA|   RangeA|AvgOfMajor N|AvgOfOth N|AvgOfNoCrim N|AvgOfProp N|AvgOfVio N|Borough Name|Postcode| Latitude| Longitude|Community Board|Council District |Census Tract|    BIN|       BBL|                 NTA|INDEX|
+-----------+-------------+------+--------------------+-------------+--------------------+-------+------------------------

In [3]:
from pyspark.sql.functions import lit, col, when, concat, avg, stddev, abs

def outlier_detection(df):

    # Create a new column to flag the outlier
    df_outlier = df.withColumn('OUTLIER', lit(''))

    # Define the list of columns that need to be checked
    outlier_columns = [column for column in df.columns if 'total' in column.lower() or 'avg' in column.lower() or 'age' in column.lower() or 'grade' in column.lower()]

    # Detect the outlier
    for column in outlier_columns:
        result = df.agg(avg(column), stddev(column)).collect()
        avg_val = result[0][0]
        stddev_val = result[0][1]
        df_outlier = df_outlier.withColumn('OUTLIER', 
                                               when((abs((col(column) - avg_val)) / stddev_val > 2.2), 
                                                            concat(col('OUTLIER'), lit(','), lit(column))).otherwise(col('OUTLIER')))
        
    # Keep only the rows that contain outlier flags
    df_outlier = df_outlier.filter(col('OUTLIER') != '')

    return df_outlier

In [4]:
# Detect outlier
df_outlier = outlier_detection(df_with_index)

df_outlier.select('INDEX', 'OUTLIER').show()

24/05/08 14:17:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+--------------------+
|INDEX|             OUTLIER|
+-----+--------------------+
|  535|,AvgOfMajor N,Avg...|
|  538|,AvgOfMajor N,Avg...|
|  553|,AvgOfMajor N,Avg...|
|  554|,AvgOfMajor N,Avg...|
|  570|,AvgOfMajor N,Avg...|
|  574|,AvgOfMajor N,Avg...|
|  595|,AvgOfMajor N,Avg...|
|  598|,AvgOfMajor N,Avg...|
|  622|,AvgOfMajor N,Avg...|
|  631|,AvgOfMajor N,Avg...|
|  873|,AvgOfMajor N,Avg...|
| 1015|,AvgOfMajor N,Avg...|
| 1038|,AvgOfMajor N,Avg...|
| 1044|,AvgOfMajor N,Avg...|
| 1055|,AvgOfMajor N,Avg...|
| 1056|,AvgOfMajor N,Avg...|
| 1065|,AvgOfMajor N,Avg...|
| 1073|,AvgOfMajor N,Avg...|
| 1093|,AvgOfMajor N,Avg...|
| 1121|,AvgOfMajor N,Avg...|
+-----+--------------------+
only showing top 20 rows



In [5]:
# Load the data stored in the file
df = spark.read.csv("dataset/Construction_Demolition_Registrants_20240413.csv", header=True, inferSchema=True)

# Trim the leading and trailing whitespaces
for column in df.columns:
    df = df.withColumn(column, trim(col(column)))

# Add a unique index to each row
df_with_index = df.withColumn('INDEX', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

df_with_index.show()

24/05/08 14:17:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+----------+--------------------+----------+--------------------+----------------+-----+----------+--------------+-----+----------------+----------------+--------------+---------------+-------+-----------+---------+----------+---------------+----------------+------------+---------+----------+--------------------+----+-----+
|   CREATED|BIC NUMBER|        ACCOUNT NAME|TRADE NAME|             ADDRESS|            CITY|STATE|  POSTCODE|         PHONE|EMAIL|APPLICATION TYPE|DISPOSITION DATE|EFFECTIVE DATE|EXPIRATION DATE|RENEWAL|EXPORT DATE| LATITUDE| LONGITUDE|COMMUNITY BOARD|COUNCIL DISTRICT|CENSUS TRACT|      BIN|       BBL|                 NTA|BORO|INDEX|
+----------+----------+--------------------+----------+--------------------+----------------+-----+----------+--------------+-----+----------------+----------------+--------------+---------------+-------+-----------+---------+----------+---------------+----------------+------------+---------+----------+--------------------+-

In [6]:
def pattern_violation_detection(df):

    # Create a new column to flag the regular expression violations
    df_pattern_violation = df.withColumn('REGEX', lit(''))

    # Define the list of columns that need to be checked
    date_columns = [column for column in df.columns if 'date' in column.lower()]
    phone_columns = [column for column in df.columns if 'phone' in column.lower()]
    email_columns = [column for column in df.columns if 'email' in column.lower()]
    zip_columns = [column for column in df.columns if 'zip' in column.lower() or 'post' in column.lower()]

    # Define the list of regular expressions
    regexp_date = r'^(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])/\d{4}$'
    regexp_phone = r'^\(\d{3}\) \d{3}-\d{4}$'
    regexp_email = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    regexp_zip = r'^\d{5}(-\d{4})?$'

    # Detect the regex pattern violation
    for column in date_columns:
        df_pattern_violation = df_pattern_violation.withColumn('REGEX', 
                                                 when(col(column).isNull() | col(column).rlike(regexp_date), 
                                                      col('REGEX')).otherwise(concat(col('REGEX'), lit(','), lit(column))))
    for column in phone_columns:
        df_pattern_violation = df_pattern_violation.withColumn('REGEX', 
                                                 when(col(column).isNull() | col(column).rlike(regexp_phone), 
                                                      col('REGEX')).otherwise(concat(col('REGEX'), lit(','), lit(column))))                                        
    for column in email_columns:
        df_pattern_violation = df_pattern_violation.withColumn('REGEX', 
                                                 when(col(column).isNull() | col(column).rlike(regexp_email), 
                                                      col('REGEX')).otherwise(concat(col('REGEX'), lit(','), lit(column))))
    for column in zip_columns:
        df_pattern_violation = df_pattern_violation.withColumn('REGEX', 
                                                 when(col(column).isNull() | col(column).rlike(regexp_zip), 
                                                      col('REGEX')).otherwise(concat(col('REGEX'), lit(','), lit(column))))
    
    # Keep only the rows that contain pattern violation flags
    df_pattern_violation = df_pattern_violation.filter((col('REGEX') != ''))

    return df_pattern_violation

In [7]:
# Detect pattern violation
df_pattern_violation = pattern_violation_detection(df_with_index)

df_pattern_violation.show()

24/05/08 14:17:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:17:09 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetric

+----------+----------+--------------------+--------------------+--------------------+----------------+-----+--------+--------------------+-----+----------------+----------------+--------------+---------------+-------+-----------+---------+----------+---------------+----------------+------------+---------+----------+--------------------+----+-----+------+
|   CREATED|BIC NUMBER|        ACCOUNT NAME|          TRADE NAME|             ADDRESS|            CITY|STATE|POSTCODE|               PHONE|EMAIL|APPLICATION TYPE|DISPOSITION DATE|EFFECTIVE DATE|EXPIRATION DATE|RENEWAL|EXPORT DATE| LATITUDE| LONGITUDE|COMMUNITY BOARD|COUNCIL DISTRICT|CENSUS TRACT|      BIN|       BBL|                 NTA|BORO|INDEX| REGEX|
+----------+----------+--------------------+--------------------+--------------------+----------------+-----+--------+--------------------+-----+----------------+----------------+--------------+---------------+-------+-----------+---------+----------+---------------+----------------+

In [8]:
from pyspark.sql.functions import struct, countDistinct

def constriant_violation_detection(df, left, right):

    # Filter the rows without null values in the left and right columns
    df_constriant_violation = df.dropna(subset=left+right)

    struct_cols_right = [col(column) for column in right]
    struct_expr_right = struct(*struct_cols_right)
    
    # Group the right columns by the left columns
    # Count the distinct values of the right columns for each group
    grouped = df_constriant_violation.groupBy(left).agg(countDistinct(struct_expr_right).alias("distinct_count"))

    # Filter the groups with more than one distinct value
    grouped = grouped.filter(col("distinct_count") > 1)

    # Join the grouped dataframe with the original dataframe
    df_constriant_violation = df_constriant_violation.join(grouped, left, "inner").select(df_constriant_violation.columns).sort(left)

    return df_constriant_violation

In [9]:
# Detect constriant violation
left = ['NTA']
right = ['BORO']
df_constriant_violation = constriant_violation_detection(df_with_index, left, right)

df_constriant_violation.select(left+right).show()

+---+-----+
|NTA| BORO|
+---+-----+
|  1|   18|
|  1|   33|
|  1|   31|
|  1|    8|
|  1|   41|
|  1|    8|
|  1|  201|
| 11|40702|
| 11|44902|
| 11|45101|
| 12|  350|
| 12|46202|
| 12|46202|
| 12|  344|
| 12|  358|
| 12|46202|
| 12|  390|
| 12|  368|
| 12|  456|
| 12|  484|
+---+-----+
only showing top 20 rows

